### Computational Guided Inquiry for Modeling Earth's Climate (Neshyba, 2025)

# Cambio 3.0

## Equations of motion of Cambio3.0
The equations of motion of Cambio3.0 are actually the same as Cambio2.0:
$$
F_{land->atm} =  k_{la} \ \ \ (1) 
$$

$$
F_{atm->land} = k_{al0} +  k_{al1} \times [C_{atm}] \ \ \ (2)
$$

$$
F_{ocean->atm} = k_{oa} \times (1+DC\times T_{anomaly}) [C_{ocean}] \ \ \ (3)
$$

$$
F_{atm->ocean} = k_{ao} [C_{atm}] \ \ \ (4)
$$

$$
F_{human->atm} = \epsilon(t) \ \ \ (5)
$$

The difference is that ice-albedo is taken into account within each iteration of the Euler loop is that after  we calculate an albedo based on the temperature anomaly, we *use that albedo to calculate a temperature anomaly*.


## Uploading your climate emissions scenario
As before, you'll need to upload a climate emissions scenario file to the current folder. 

## Learning goals
1. I can describe the geophysical processes behind ice-albedo feedback.
1. I can implement ice-albedo feedback in an Euler loop.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meclib.cl as cl
from copy import copy as makeacopy

In [ ]:
%matplotlib notebook

### Loading the scheduled flow
In the cell below, load in your scheduled flows file. It'll be most convenient if you use the following naming convention: 

    time, eps, epsdictionary_fromfile = cl.LoadMyScenario(...)


(but of course supplying the name of your own scheduled flows file).

In [ ]:
# your code here 


### Creating a dictionary for climate parameters
In the cell below, we use the CreateClimateParams function to create a dictionary of climate parameters.

In [ ]:
ClimateParams = cl.CreateClimateParams(epsdictionary_fromfile)
display(ClimateParams)

### Duplicating Cambio2.0
Below, we reproduce Cambio2.0.

In [ ]:
def PropagateCS_Cambio2(previousClimateState, ClimateParams, dt, F_ha):
    """Propagates the state of the climate, with a specified anthropogenic carbon flux"""
    """Returns a new climate state"""

    # Extract constants from ClimateParams
    k_la = ClimateParams['k_la']
    k_al0 = ClimateParams['k_al0']
    k_al1 = ClimateParams['k_al1']
    k_oa = ClimateParams['k_oa']
    k_ao = ClimateParams['k_ao']
    DC = ClimateParams['DC']
    preindustrial_albedo = ClimateParams['preindustrial albedo']
    fractional_albedo_floor = ClimateParams['fractional_albedo_floor']
    albedo_Tstar = ClimateParams['albedo_Tstar']
    albedo_delta_T = ClimateParams['albedo_delta_T']
    k_al1_Tstar = ClimateParams['k_al1_Tstar']
    k_al1_deltaT = ClimateParams['k_al1_deltaT']
    fractional_k_al1_floor = ClimateParams['fractional_k_al1_floor']
    
    # Extract concentrations, albedo, etc, from the previous climate state
    C_atm = previousClimateState['C_atm']
    C_ocean = previousClimateState['C_ocean']
    albedo = previousClimateState['albedo']
    time = previousClimateState['time']
    
    # Get the temperature implied by the carbon in the atmosphere, and ocean pH
    T_anomaly = cl.Diagnose_T_anomaly(C_atm, preindustrial_albedo, ClimateParams)
    actual_temperature = cl.Diagnose_actual_temperature(T_anomaly)
    OceanSurfacepH = cl.Diagnose_OceanSurfacepH(C_atm,ClimateParams)
    
    # Get new fluxes (including the effect of temperature anomaly on the ocean-to-atmosphere flux)
    F_la = k_la    
    F_al = k_al0 + k_al1*C_atm
    F_oa = k_oa*C_ocean*(1+DC*T_anomaly)    
    F_ao = k_ao*C_atm

    # Get new concentrations of carbon that depend on the fluxes
    C_atm += (F_la + F_oa - F_ao - F_al + F_ha)*dt
    C_ocean += (F_ao - F_oa)*dt
    time += dt
    
    # Create a new climate state with these updates
    ClimateState = makeacopy(previousClimateState)
    ClimateState['C_atm'] = C_atm
    ClimateState['C_ocean'] = C_ocean
    ClimateState['F_al'] = F_al
    ClimateState['F_la'] = F_la
    ClimateState['F_ao'] = F_ao
    ClimateState['F_oa'] = F_oa
    ClimateState['F_ha'] = F_ha
    ClimateState['F_ocean_net'] = F_oa-F_ao
    ClimateState['F_land_net'] = F_la-F_al
    ClimateState['time'] = time
    ClimateState['T_anomaly'] = T_anomaly
    ClimateState['actual temperature'] = actual_temperature
    ClimateState['OceanSurfacepH'] = OceanSurfacepH
    ClimateState['albedo'] = albedo

    # Return the new climate state
    return ClimateState

# Run Cambio2.0
CS_Cambio2_list = cl.run_Cambio(PropagateCS_Cambio2, ClimateParams, time, eps)

# Choose items to plot
items_to_plot = [['C_atm','C_ocean'],['F_ha','F_ocean_net','F_land_net'],'T_anomaly','albedo','OceanSurfacepH']

# Plot those items
cl.CS_list_plots(CS_Cambio2_list,'Cambio2',items_to_plot)

### Cambio 3.0
Below, the goal is to enhance Cambio2.0 with ice-albedo feedback. It'll be mostly a cut-and-paste job, but you'll need to make an adjustment in the function that propagates the climate state forward. Then make all the plots you did for Cambio2.0.

In [ ]:
# your code here 


### Pause for analysis
Can you explain why there's such a big difference in the peak temperature between Cambio2.0 and Cambio3.0?

YOUR ANSWER HERE

### Validating and finishing up
Assuming all this has gone smoothly, don't forget to do a Kernel/Restart & Run All, run the whole notebook, and make sure there aren't any errors.